In [83]:
import numpy as np
import pandas as pd
import re
import statistics

In [84]:
df = pd.read_csv("forR_input.csv", dtype=str)
df = df.apply(lambda col: col.str.lower() if col.dtype == 'object' else col)

df = df[df['STRING'].isin(['all', 'cat', 'con', 'hat'])]

df


,STRING,initial1,initial1.1,initial1.2,1a,1a.1,1a.2,1b,1b.1,1b.2,...,initial6.2,6a,6a.1,6a.2,6b,6b.1,6b.2,6c,6c.1,6c.2
218,all,ɔ,a,ɔ+a,l,ll,l+ll,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1546,cat,k,c,k+c,æ,a,æ+a,t,t,t+t,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2028,con,k,c,k+c,ɑ,o,ɑ+o,n,n,n+n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4422,hat,h,h,h+h,æ,a,æ+a,t,t,t+t,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
df.insert(1,'complexity', 0)
df

,STRING,complexity,initial1,initial1.1,initial1.2,1a,1a.1,1a.2,1b,1b.1,...,initial6.2,6a,6a.1,6a.2,6b,6b.1,6b.2,6c,6c.1,6c.2
218,all,0,ɔ,a,ɔ+a,l,ll,l+ll,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1546,cat,0,k,c,k+c,æ,a,æ+a,t,t,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2028,con,0,k,c,k+c,ɑ,o,ɑ+o,n,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4422,hat,0,h,h,h+h,æ,a,æ+a,t,t,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
# pg_df = pd.DataFrame()
# all_phonemes = df['initial1'].tolist() + df['1a'].tolist() + df['1b'].tolist()
# all_phonemes = list(set(all_phonemes))
# pg_df['phoneme'] = all_phonemes

# for phoneme in all_phonemes:
#     filtered_phoneme_df = df.loc[(df['initial1'] == phoneme) | (df['1a'] == phoneme) | (df['1b'] == phoneme)]
#     all_graphemes_for_phoneme_list = filtered_phoneme_df['initial1.1'].tolist() + filtered_phoneme_df['1a.1'].tolist() + filtered_phoneme_df['1b.1'].tolist()
#     all_graphemes_for_phoneme_set = set(all_graphemes_for_phoneme_list)
#     all_graphemes_for_phoneme_map = {grapheme: all_graphemes_for_phoneme_list.count(grapheme) for grapheme in all_graphemes_for_phoneme_set}
#     pg_df.loc[pg_df['phoneme'] == phoneme, 'grapheme_map'] = all_graphemes_for_phoneme_map
#     pg_df.loc[pg_df['phoneme'] == phoneme, 'grapheme_count'] = len(filtered_phoneme_df.index)

# pg_df


# do not use this part:
# all_graphemes = df['initial1.1'].tolist() + df['1a.1'].tolist() + df['1b.1'].tolist()
# pg_map = {grapheme: 0 for grapheme in all_graphemes}
# pg_df['pg_mappings'] = pg_map
# pg_df

In [87]:
pg_df = pd.DataFrame()
all_phonemes = df['initial1'].tolist() + df['1a'].tolist() + df['1b'].tolist()
all_graphemes = df['initial1.1'].tolist() + df['1a.1'].tolist() + df['1b.1'].tolist()
all_pg_pairs = df['initial1.2'].tolist() + df['1a.2'].tolist() + df['1b.2'].tolist()
pg_df['phoneme'] = all_phonemes
pg_df['grapheme'] = all_graphemes
pg_df['pg_pair'] = all_pg_pairs
pg_df = pg_df.dropna()


# unique_phonemes = set(all_phonemes)
# for phoneme in unique_phonemes:
#     filtered_phoneme_df = pg_df.loc[(pg_df['phoneme'] == phoneme)]
#     pg_df.loc[pg_df['phoneme'] == phoneme, 'instances_of_phoneme'] = len(filtered_phoneme_df.index)

# unique_graphemes = set(all_graphemes)
# for grapheme in unique_graphemes:
#     filtered_grapheme_df = pg_df.loc[(pg_df['grapheme'] == grapheme)]
#     pg_df.loc[pg_df['grapheme'] == grapheme, 'instances_of_grapheme'] = len(filtered_grapheme_df.index)
    
pg_df

,phoneme,grapheme,pg_pair
0,ɔ,a,ɔ+a
1,k,c,k+c
2,k,c,k+c
3,h,h,h+h
4,l,ll,l+ll
5,æ,a,æ+a
6,ɑ,o,ɑ+o
7,æ,a,æ+a
9,t,t,t+t
10,n,n,n+n


In [91]:
def conditional_prob_p_to_g(p, g):
    p_g_instances = len((pg_df.loc[(pg_df['phoneme'] == p) & (pg_df['grapheme'] == g)]).index)
    p_instances = len((pg_df.loc[(pg_df['phoneme'] == p)]).index)
    return p_g_instances / p_instances

def conditional_prob_g_to_p(g, p):
    p_g_instances = len((pg_df.loc[(pg_df['phoneme'] == p) & (pg_df['grapheme'] == g)]).index)
    g_instances = len((pg_df.loc[(pg_df['grapheme'] == g)]).index)
    return p_g_instances / g_instances

def p_plus_g_frequency(p, g):
    p_g_instances = len((pg_df.loc[(pg_df['phoneme'] == p) & (pg_df['grapheme'] == g)]).index)
    all_instances = len(pg_df.index)
    return p_g_instances / all_instances

print("p->g for 'ɔ' to 'a':", conditional_prob_p_to_g('ɔ', 'a'))
print("g->p for 'a' to 'ɔ':", conditional_prob_g_to_p('a', 'ɔ'))
print("ɔ+a frequency:", p_plus_g_frequency('ɔ', 'a'))
    


p->g for 'ɔ' to 'a': 1.0
g->p for 'a' to 'ɔ': 0.3333333333333333
ɔ+a frequency: 0.09090909090909091


In [93]:
p_input = input("Enter phoneme:\n")
g_input = input("Enter grapheme:\n")
option = input("OPTIONS:\n1 for p->g conditional probability.\n2 for g->p conditional probability.\n3 for p+g frequency.\nEnter option:")


In [94]:
if option == '1':
    print(conditional_prob_p_to_g(p_input, g_input))
elif option == '2':
    print(conditional_prob_g_to_p(g_input, p_input))
elif option == '3':
    print(p_plus_g_frequency(p_input, g_input))

1.0


In [95]:
# P->G PHONOGRAPHEME CONSISTENCY

def calculate_p_to_g_phonographeme_probs(word):
    word = word.lower()
    p_to_g_probs = []
    if word in df['STRING'].values:
        word_phonemes = [df.loc[(df['STRING'] == word), 'initial1'].values[0], df.loc[(df['STRING'] == word), '1a'].values[0], df.loc[(df['STRING'] == word), '1b'].values[0]]
        word_phonemes = [x for x in word_phonemes if str(x) != 'nan']
        word_graphemes = [df.loc[(df['STRING'] == word), 'initial1.1'].values[0], df.loc[(df['STRING'] == word), '1a.1'].values[0], df.loc[(df['STRING'] == word), '1b.1'].values[0]]
        word_graphemes = [x for x in word_graphemes if str(x) != 'nan']
        for index in range(len(word_phonemes)):
            p_to_g_probs.append(conditional_prob_p_to_g(word_phonemes[index], word_graphemes[index]))
    return p_to_g_probs
    
def p_to_g_phonographeme_consistency_median(word):
    p_to_g_probs = calculate_p_to_g_phonographeme_probs(word)
    if len(p_to_g_probs) != 0:
        return statistics.median(p_to_g_probs)
    return None

def p_to_g_phonographeme_consistency_mean(word):
    p_to_g_probs = calculate_p_to_g_phonographeme_probs(word)
    if len(p_to_g_probs) != 0:
        return statistics.mean(p_to_g_probs)
    return None

def p_to_g_phonographeme_consistency_max(word):
    p_to_g_probs = calculate_p_to_g_phonographeme_probs(word)
    if len(p_to_g_probs) != 0:
        return max(p_to_g_probs)
    return None

def p_to_g_phonographeme_consistency_min(word):
    p_to_g_probs = calculate_p_to_g_phonographeme_probs(word)
    if len(p_to_g_probs) != 0:
        return min(p_to_g_probs)
    return None

def p_to_g_phonographeme_consistency_distribution_quantile(word, q): # e.g. q = 0.25 for Q1, q = 0.7 for 70th percentile
    p_to_g_probs = calculate_p_to_g_phonographeme_probs(word)
    if len(p_to_g_probs) != 0 and q >= 0 and q <= 100:
        return np.quantile(p_to_g_probs, q)
    return None

def p_to_g_phonographeme_consistency_IQR(word):
    return p_to_g_phonographeme_consistency_distribution_quantile(word, 0.75) - p_to_g_phonographeme_consistency_distribution_quantile(word, 0.25)

def p_to_g_phonographeme_consistency_range(word):
    return p_to_g_phonographeme_consistency_max(word) - p_to_g_phonographeme_consistency_min(word)

In [97]:
word_input = input("P->G PHONOGRAPHEME CONSISTENCY\nEnter word:\n")
if word_input.lower() not in df['STRING'].values:
    print("Word not found. Exiting.")
else:
    p_to_g_photographeme_consistency_option = input("OPTIONS:\n1 for median.\n2 for mean.\n3 for max.\n4 for min.\nEnter option:")
    
    if p_to_g_photographeme_consistency_option == '1':
        print(p_to_g_phonographeme_consistency_median(word_input))
    elif p_to_g_photographeme_consistency_option == '2':
        print(p_to_g_phonographeme_consistency_mean(word_input))
    elif p_to_g_photographeme_consistency_option == '3':
        print(p_to_g_phonographeme_consistency_max(word_input))
    elif p_to_g_photographeme_consistency_option == "4":
        print(p_to_g_phonographeme_consistency_min(word_input))



1.0


In [ ]:
# G->P PHONOGRAPHEME CONSISTENCY

def calculate_g_to_p_phonographeme_probs(word):
    word = word.lower()
    g_to_p_probs = []
    if word in df['STRING'].values:
        word_phonemes = [df.loc[(df['STRING'] == word), 'initial1'].values[0], df.loc[(df['STRING'] == word), '1a'].values[0], df.loc[(df['STRING'] == word), '1b'].values[0]]
        word_phonemes = [x for x in word_phonemes if str(x) != 'nan']
        word_graphemes = [df.loc[(df['STRING'] == word), 'initial1.1'].values[0], df.loc[(df['STRING'] == word), '1a.1'].values[0], df.loc[(df['STRING'] == word), '1b.1'].values[0]]
        word_graphemes = [x for x in word_graphemes if str(x) != 'nan']
        for index in range(len(word_phonemes)):
            g_to_p_probs.append(conditional_prob_g_to_p(word_graphemes[index], word_phonemes[index]))
    return g_to_p_probs
    
def g_to_p_phonographeme_consistency_median(word):
    g_to_p_probs = calculate_g_to_p_phonographeme_probs(word)
    if len(g_to_p_probs) != 0:
        return statistics.median(g_to_p_probs)
    return None

def g_to_p_phonographeme_consistency_mean(word):
    g_to_p_probs = calculate_g_to_p_phonographeme_probs(word)
    if len(g_to_p_probs) != 0:
        return statistics.mean(g_to_p_probs)
    return None

def g_to_p_phonographeme_consistency_max(word):
    g_to_p_probs = calculate_g_to_p_phonographeme_probs(word)
    if len(g_to_p_probs) != 0:
        return max(g_to_p_probs)
    return None

def g_to_p_phonographeme_consistency_min(word):
    g_to_p_probs = calculate_g_to_p_phonographeme_probs(word)
    if len(g_to_p_probs) != 0:
        return min(g_to_p_probs)
    return None

def g_to_p_phonographeme_consistency_distribution_quantile(word, q): # e.g. q = 0.25 for Q1, q = 0.7 for 70th percentile
    g_to_p_probs = calculate_g_to_p_phonographeme_probs(word)
    if len(g_to_p_probs) != 0 and q >= 0 and q <= 100:
        return np.quantile(g_to_p_probs, q)
    return None

def g_to_p_phonographeme_consistency_IQR(word):
    return g_to_p_phonographeme_consistency_distribution_quantile(word, 0.75) - g_to_p_phonographeme_consistency_distribution_quantile(word, 0.25)

def p_to_g_phonographeme_consistency_range(word):
    return g_to_p_phonographeme_consistency_max(word) - g_to_p_phonographeme_consistency_min(word)

In [ ]:
word_input = input("G->P PHONOGRAPHEME CONSISTENCY\nEnter word:\n")
if word_input.lower() not in df['STRING'].values:
    print("Word not found. Exiting.")
else:
    g_to_p_photographeme_consistency_option = input("OPTIONS:\n1 for median.\n2 for mean.\n3 for max.\n4 for min.\nEnter option:")
    
    if g_to_p_photographeme_consistency_option == '1':
        print(g_to_p_phonographeme_consistency_median(word_input))
    elif g_to_p_photographeme_consistency_option == '2':
        print(g_to_p_phonographeme_consistency_mean(word_input))
    elif g_to_p_photographeme_consistency_option == '3':
        print(g_to_p_phonographeme_consistency_max(word_input))
    elif g_to_p_photographeme_consistency_option == "4":
        print(g_to_p_phonographeme_consistency_min(word_input))